In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
def insert_element_randomly(my_list, element):
    
    if len(my_list) > 1:

        index = random.randint(0, 2)
        
    else:
        
        index = 0

    new_list = my_list[:(4 * index)] + element + my_list[(4 * index):]
    
    return new_list

In [7]:
N = 20 # vocab_size
M = 20 # number of random words

vocabs = ['word_' + str(i) for i in range(N)] + ['random_' + str(i) for i in range(M)]

vocabs_word = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i

In [8]:
def get_accuracy_prob(embed_dim):
    
    pairs = []

    for i in vocabs_word:
        for j in vocabs_word:
            for k in vocabs_word:
                if i != j and i != k and j != k:
                    pairs.append((i,j,k))

    indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

    pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
    pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]
    
    sentences_train = []
    sentences_number_train = []
    sentences_test_a = []
    sentences_number_test_a = []
    sentences_test_b = []
    sentences_number_test_b = []

    x_masked_train = []
    y_masked_labels_train = []
    x_masked_test_a = []
    y_masked_labels_test_a = []
    x_masked_test_b = []
    y_masked_labels_test_b = []

    for _ in range(25000):

        random_words = random.sample(['random_' + str(i) for i in range(M)], 4)
    
        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, a, d, e, f, d]
        temp = insert_element_randomly(temp, random_words)

        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])
        
        random_words = random.sample(['random_' + str(i) for i in range(M)], 4)

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, b, d, e, f, e]
        temp = insert_element_randomly(temp, random_words)

        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])



    for _ in range(25000):

        [(a,b,c), (d,e,f), (g,h,i)] = random.sample(pairs_test, 3)
    
        temp = [a, b, c, a, d, e, f, d, g, h, i, g]

        sentences_test_a.append(temp)
        sentences_number_test_a.append([vocab_map[i] for i in temp])
        x_masked_test_a.append([vocab_map[i] for i in temp])
        y_masked_labels_test_a.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f), (g,h,i)] = random.sample(pairs_test, 3)
    
        temp = [a, b, c, b, d, e, f, e, g, h, i, h]

        sentences_test_b.append(temp)
        sentences_number_test_b.append([vocab_map[i] for i in temp])
        x_masked_test_b.append([vocab_map[i] for i in temp])
        y_masked_labels_test_b.append([vocab_map[i] for i in temp][1:])

    x_masked_train = np.array(x_masked_train)
    y_masked_labels_train = np.array(y_masked_labels_train)
    x_masked_test_a = np.array(x_masked_test_a)
    y_masked_labels_test_a = np.array(y_masked_labels_test_a)
    x_masked_test_b = np.array(x_masked_test_b)
    y_masked_labels_test_b = np.array(y_masked_labels_test_b)

    perm = np.random.permutation(len(x_masked_train))
    x_masked_train = x_masked_train[perm]
    y_masked_labels_train = y_masked_labels_train[perm]
    
    num_head = 2

    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(N + M, embed_dim, name="word_embedding")(inputs)
    sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
    encoder_output = word_embeddings + sinusoidal_embeddings
    for i in range(5):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

    encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
    mlm_output = layers.Dense(N + M, name="mlm_cls", activation="softmax")(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                            validation_split = 0.5, callbacks = [callback], 
                            epochs=2000, batch_size=5000, 
                            verbose=0)
    
    acc_a = []
    prob_a = []
    x_test_subset_a = x_masked_test_a[np.random.choice(x_masked_test_a.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_a:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_a.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_a.append(temp[0][sentence_number[-1]])
        
    acc_b = []
    prob_b = []
    x_test_subset_b = x_masked_test_b[np.random.choice(x_masked_test_b.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_b:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_b.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_b.append(temp[0][sentence_number[-1]])
        
    return ((np.mean(acc_a), np.mean(prob_a)), (np.mean(acc_b), np.mean(prob_b)))


In [10]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(10)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

(0.071, 0.04972046)
(0.051, 0.049636543)
(0.055, 0.04862298)
(0.048, 0.049383644)
(0.054, 0.05255547)
(0.075, 0.062781446)
(0.0, 0.00022943396)
(0.0, 0.00022697105)
(0.0, 0.0012989511)
(0.0, 0.0012571892)
(0.021, 0.022933366)
(0.021, 0.022641309)
(0.05, 0.050291426)
(0.064, 0.050236445)
(0.052, 0.050508823)
(0.048, 0.05122985)
(0.04, 0.05071496)
(0.053, 0.050393622)
(0.053, 0.048803724)
(0.034, 0.048629038)
(0.0396, 0.037567959443549626)
(0.0394, 0.03864160561352037)


In [9]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

2024-05-04 00:17:08.503406: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-04 00:17:08.503423: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-04 00:17:08.503443: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3413.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-04 00:17:08.528454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(0.0, 7.0075555e-05)
(0.0, 6.910433e-05)
(0.031, 0.018794963)
(0.046, 0.02544874)
(0.253, 0.2013696)
(0.445, 0.31731474)
(0.033, 0.036250047)
(0.058, 0.03651711)
(0.057, 0.048197564)
(0.059, 0.048549335)
(0.0, 6.954224e-05)
(0.0, 6.8852365e-05)
(0.049, 0.03641667)
(0.06, 0.03693294)
(0.0, 0.00013894202)
(0.0, 0.00014568478)
(0.0, 9.9120836e-05)
(0.0, 0.00010075606)
(0.013, 0.015946135)
(0.019, 0.01457485)
(0.04360000000000001, 0.035735265679250006)
(0.0687, 0.047972211773594606)
